# **Twitter Sentiment Analysis** 

### Understanding Brand Perception Through Natural Language Processing

## Business Problem and Context

In today's digital marketplace, social media has become the primary channel where customers express their opinions about products and brands. For companies like Apple and Google, understanding public sentiment on Twitter can provide invaluable insights into:

- **Product reception**: How are new product launches being received?
- **Brand health**: What's the overall sentiment toward our brand vs competitors?
- **Crisis detection**: Can we identify emerging negative sentiment before it becomes a PR crisis?
- **Customer service prioritization**: Which complaints need immediate attention?

**The Challenge**: Manually analyzing thousands of tweets daily is impossible. Customer service teams are overwhelmed, and by the time negative sentiment is identified through traditional methods, brand damage may already be done.

**Our Solution**: Build an automated sentiment classification system that can process Twitter data in real-time, categorizing tweets as positive, negative, neutral, or uncertain. This system will help stakeholders:

1. Monitor brand health continuously
2. Identify trending issues early
3. Route negative sentiment to customer service teams
4. Measure campaign effectiveness

**Success Metrics**: We aim to build a model that can accurately classify tweet sentiment with high precision and recall, particularly for negative sentiment (where misclassification is most costly from a business perspective).

## Table of Contents

1. [Business Understanding](#business-problem-and-context)
2. [Data Understanding](#data-understanding)
3. [Data Preparation](#data-preparation)
4. [Exploratory Data Analysis](#exploratory-data-analysis)
5. [Text Preprocessing & Feature Engineering](#text-preprocessing-and-feature-engineering)
6. [Modeling](#modeling)
7. [Model Evaluation & Interpretation](#model-evaluation-and-interpretation)
8. [Conclusions & Recommendations](#conclusions-and-recommendations)

## Data Understanding

Our dataset consists of tweets collected during the South by Southwest (SXSW) conference, where Apple and Google products were prominently discussed. Each tweet has been manually labeled by human judges for sentiment toward specific brands or products.

This is a real-world dataset with all its messiness: typos, slang, emojis, hashtags, and the informal language typical of social media. Understanding this data is our first critical step.


### Importing Libraries

In [3]:
# Core data manipulation and analysis
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Text processing and NLP
import re
import string
from collections import Counter

# NLTK for advanced NLP tasks (required by rubric)
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams

# Download required NLTK data
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('punkt_tab', quiet=True)

# Scikit-learn for machine learning
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    classification_report, confusion_matrix, 
    accuracy_score, precision_recall_fscore_support
)

# For model persistence
import joblib

# Set random seed for reproducibility
np.random.seed(42)

### Loading and Initial Exploration

In [5]:
df = pd.read_csv("../data/judge-1377884607_tweet_product_company.csv",encoding="latin1")
df.head()


# basic information
print("Dataset Shape:", df.shape)
print("\n" + " "*5)
print("Column Names:")
print(" "*3)
for col in df.columns:
    print(f"  - {col}")

print("\n" + " "*60)
print("First Few Rows:")
print(" "*6)
df.head()

Dataset Shape: (9093, 3)

     
Column Names:
   
  - tweet_text
  - emotion_in_tweet_is_directed_at
  - is_there_an_emotion_directed_at_a_brand_or_product

                                                            
First Few Rows:
      


,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


### Rename columns for clarity and ease of use

In [6]:
df.columns = ['tweet', 'product', 'sentiment']

print("Renamed columns:")
df.head(3)


Renamed columns:


,tweet,product,sentiment
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
